# Demonstration of AEquitas

Hello, in this notebook, we will demonstrate AEquitas on a sample dataset!

We will be using the Obermeiyer dataset as an example of how AEquitas can both be used for social bias detection!

In [2]:
import pandas as pd
from cnnMCSE.dissecting.util.features import *
from cnnMCSE.dissecting.util.main import *
from cnnMCSE.dissecting.util.model import *
from sklearn.preprocessing import MinMaxScaler


In [3]:
#
path = '/sc/arion/projects/EHR_ML/gulamf01/dissecting-bias/data/data_new.csv'

In [4]:
metadata_df = pd.read_csv(path)
metadata_df.reset_index(inplace=True)
metadata_df.head()

,index,risk_score_t,program_enrolled_t,cost_t,cost_avoidable_t,bps_mean_t,ghba1c_mean_t,hct_mean_t,cre_mean_t,ldl_mean_t,...,trig_min-high_tm1,trig_min-normal_tm1,trig_mean-low_tm1,trig_mean-high_tm1,trig_mean-normal_tm1,trig_max-low_tm1,trig_max-high_tm1,trig_max-normal_tm1,gagne_sum_tm1,gagne_sum_t
0,0,1.987430,0,1200.0,0.0,NaN,5.4,NaN,1.110000,194.0,...,0,0,0,0,0,0,0,0,0,0
1,1,7.677934,0,2600.0,0.0,119.0,5.5,40.4,0.860000,93.0,...,0,1,0,0,1,0,0,1,4,3
2,2,0.407678,0,500.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,3,0.798369,0,1300.0,0.0,117.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4,17.513165,0,1100.0,0.0,116.0,NaN,34.1,1.303333,53.0,...,0,0,0,0,0,0,0,0,1,1


In [5]:
metadata_df, x_column_names, Y_predictors = get_Y_x_df(metadata_df, verbose=True)

Features breakdown:
   demographic: 8
   comorbidity: 34
   cost: 13
   lab: 90
   med: 4
 TOTAL: 149


In [6]:
def assign_race(value):
    if(value == 0): return "White"
    else: return "Black"

In [8]:
metadata_df['demographics'] = metadata_df['dem_race_black'].apply(assign_race)
metadata_df.drop(['dem_race_black'], axis=1, inplace=True)

In [9]:
Y_predictors
outcome = 'log_cost_t'
threshold=0.5
thresholds=None

In [10]:
scaler = MinMaxScaler()
X_vars = metadata_df[x_column_names]
X_vars = scaler.fit_transform(X_vars.to_numpy())
X_df = pd.DataFrame(X_vars, columns=x_column_names)

In [11]:
def label_encoding(y_df:pd.DataFrame, outcome:str, threshold:float=None, thresholds:list=None):
    """Encode labels in a metadata file. 

    Args:
        metadata_df (pd.DataFrame): Metadata data frame generated synthetically. 
        outcome (str): Outcome value. 
        threshold (float): Threshold to classify. 
    """
    if(threshold):
        y_df['target'] = (y_df[outcome] > y_df[outcome].quantile(q=threshold)).apply(int)
        y_df['target_map'] = np.where(y_df['target'] == 1, 'high_risk', 'low_risk')
    
    if(thresholds):
        y_df['target'] = 0
        for index, threshold in enumerate(thresholds):
            y_df.loc[(y_df[outcome] > y_df[outcome].quantile(q=threshold)), ['target']] = index 
        y_df['target_map'] = np.where(y_df['target'] > 5, 'high_risk', 'low_risk')

    return y_df

In [12]:
Y_predictors

['log_cost_t', 'gagne_sum_t', 'log_cost_avoidable_t']

In [13]:
# Label encoding. 

threshold=0.5
thresholds=None
y_df = metadata_df[Y_predictors]
outcome = 'log_cost_t'
y_df = label_encoding(y_df, outcome, threshold=threshold, thresholds=thresholds)
y_df = y_df.drop(Y_predictors, axis=1)
X_df[outcome] = list(y_df['target'])


y_df = metadata_df[Y_predictors]
outcome = 'gagne_sum_t'
y_df = label_encoding(y_df, outcome, threshold=threshold, thresholds=thresholds)
y_df = y_df.drop(Y_predictors, axis=1)
X_df[outcome] = list(y_df['target'])


y_df = metadata_df[Y_predictors]
outcome = 'log_cost_avoidable_t'
y_df = label_encoding(y_df, outcome, threshold=threshold, thresholds=thresholds)
y_df = y_df.drop(Y_predictors, axis=1)
X_df[outcome] = list(y_df['target'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [14]:
X_df.head()

,dem_female,dem_age_band_18-24_tm1,dem_age_band_25-34_tm1,dem_age_band_35-44_tm1,dem_age_band_45-54_tm1,dem_age_band_55-64_tm1,dem_age_band_65-74_tm1,dem_age_band_75+_tm1,alcohol_elixhauser_tm1,anemia_elixhauser_tm1,...,trig_max-low_tm1,trig_max-high_tm1,trig_max-normal_tm1,lasix_dose_count_tm1,lasix_min_daily_dose_tm1,lasix_mean_daily_dose_tm1,lasix_max_daily_dose_tm1,log_cost_t,gagne_sum_t,log_cost_avoidable_t
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1,0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [15]:
X_df['demographics'] = metadata_df['demographics']

In [16]:
X_df.to_csv("/sc/arion/projects/EHR_ML/gulamf01/cnnMCSE/demo/preprocssed_demo_demographics.tsv", sep="\t", index=False)